In [ ]:
#make dirs for raw snapper pooled files
for LANE in {1..8}
do
mkdir /workspace/hramzr/2_Phd_PROJECT/GBS/raw_snap/lane${LANE}
done

In [ ]:
#fill dirs with raw snapper pooled files
sn_gbs=/input/genomic/fish/Chrysophrys/auratus/GBS/2015
for LANE in {1..8} 
do 
bsub \
-e rs.e -J "RS" \
"cp ${sn_gbs}/*L00${LANE}* /workspace/hramzr/2_Phd_PROJECT/GBS/raw_snap/lane${LANE}"
done


In [ ]:
#Get keyfile
cp /workspace/hrajpb/Snapper/stacks/barcodes/Key_file.csv /workspace/hramzr/2_Phd_PROJECT/GBS/barcodes 

In [ ]:
#deal with duplicate IDS by adding a .1, .2 if detected
counter=0
new = open("/workspace/hramzr/2_Phd_PROJECT/GBS/barcodes/key_file_unique.txt", "w")
with open('/workspace/hramzr/2_Phd_PROJECT/GBS/barcodes/Key_file.csv') as f:
    next(f)
    content = f.readlines()
    IDS=[]
    for line in content:
        counter+=1
        line=line.strip()
        line=line.split(",")
        if line[7] in IDS:
            new.write(line[0]+"\t"+line[6]+"\t"+line[7]+"_"+str(IDS.count(line[7]))+"\n")
        else:
            new.write(line[0]+"\t"+line[6]+"\t"+line[7]+"\n")
        IDS.append(line[7])
new.close()

In [ ]:
#create barcode file

bcode_dir=/workspace/hramzr/2_Phd_PROJECT/GBS/barcodes/
keyfile_uniq=/workspace/hramzr/2_Phd_PROJECT/GBS/barcodes/key_file_unique.txt

for N in {1..8}
do
cat $keyfile_uniq | awk -v N=$N -F '\t' '{if($1==N){print $2"\t"$3}}' >/workspace/hramzr/2_Phd_PROJECT/GBS/barcodes/sn_barcode_${N}
done

In [ ]:
#check amount of individuals == checks out
cat /workspace/hramzr/2_Phd_PROJECT/GBS/barcodes/sn_barcode* | awk '{print $2}' | sort | uniq -c | wc -l
# cat /workspace/hramzr/2_Phd_PROJECT/GBS/barcodes/sn_barcode*

In [ ]:
module load stacks/2.2
for LANE in {1..8}
do
bsub \
-o GBS/stacks_log/snapper/sn_radtags_${LANE}.out -e GBS/stacks_log/snapper/sn_radtags_${LANE}.err -J "SN_RT lane ${LANE}" \
"process_radtags \
-p /powerplant/workspace/hramzr/2_Phd_PROJECT/GBS/raw_snap/lane${LANE} \
-b GBS/barcodes/sn_barcode_${LANE} \
-o GBS/indv_snapper/ \
--renz_1 'pstI' \
--renz_2 'mspI' \
-r"
done

In [ ]:
#concacetate files that are duplicates and triplicates and store all duplicate and triplicates as singletons
ls GBS/indv_snapper/*_*.gz | tr -d [a-zA-Z_./] | sed 's/.$//' | sort | uniq -c |
awk '{print $2}'>/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/multi_files

# grep all of the snapper indvs without duplicates and triplicates
ls /workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper | egrep "*.gz" | tr -d [a-z.] | egrep -v "_">/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/indv_files

# find the difference in files that are concacetates and those who aren't
grep -Fvf   /workspace/hramzr/2_Phd_PROJECT/GBS/tmp/multi_files  /workspace/hramzr/2_Phd_PROJECT/GBS/tmp/indv_files | awk '{print $0".fq.gz"}' \
>/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/missing

while read -r file_ID
do
bsub \
-o GBS/stacks_log/concacetate/multi.out -e GBS/stacks_log/concacetate/multi.err -J "concacetate" \
"cat /workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper/*${file_ID}*> \
/workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper_conc/${file_ID}.fq.gz"
done < /workspace/hramzr/2_Phd_PROJECT/GBS/tmp/multi_files

while read -r file_id
do
bsub \
-o GBS/stacks_log/concacetate/cp.out -e GBS/stacks_log/concacetate/cp.err -J "concacetate" \
"cp /workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper/*${file_id}* /workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper_conc/"
done < /workspace/hramzr/2_Phd_PROJECT/GBS/tmp/missing

In [ ]:
cat /workspace/hrajpb/Snapper/stacks/barcodes/Key_file.csv | awk -F ',' '{print $8}' | sort | uniq -c | wc -l
cat /workspace/hramzr/2_Phd_PROJECT/GBS/barcodes/Key_file.csv | awk -F ',' '{print $8}' | sort | uniq -c 

In [ ]:
find /workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper/ | egrep "fq" | egrep -v "_1" | egrep -v "_2" | awk -F '/' '{print $0, $NF}'>/workspace/hramzr/2_Phd_PROJECT/GBS/tmp/purge

In [ ]:
cat /workspace/hramzr/2_Phd_PROJECT/GBS/tmp/purge

In [3]:
while read -r file filed
do
bsub \
-o GBS/stacks_log/concacetate/cp.out -e GBS/stacks_log/concacetate/cp.err -J "concacetate" \
"cp  $file /workspace/hramzr/2_Phd_PROJECT/GBS/indv_snapper_purged"
done < /workspace/hramzr/2_Phd_PROJECT/GBS/tmp/purge

Job <276272> is submitted to default queue <lowpriority>.
Job <276273> is submitted to default queue <lowpriority>.
Job <276274> is submitted to default queue <lowpriority>.
Job <276275> is submitted to default queue <lowpriority>.
Job <276276> is submitted to default queue <lowpriority>.
Job <276277> is submitted to default queue <lowpriority>.
Job <276278> is submitted to default queue <lowpriority>.
Job <276279> is submitted to default queue <lowpriority>.
Job <276280> is submitted to default queue <lowpriority>.
Job <276281> is submitted to default queue <lowpriority>.
Job <276282> is submitted to default queue <lowpriority>.
Job <276283> is submitted to default queue <lowpriority>.
Job <276284> is submitted to default queue <lowpriority>.
Job <276285> is submitted to default queue <lowpriority>.
Job <276286> is submitted to default queue <lowpriority>.
Job <276287> is submitted to default queue <lowpriority>.
Job <276288> is submitted to default queue <lowpriority>.
Job <276289> i

Job <276414> is submitted to default queue <lowpriority>.
Job <276415> is submitted to default queue <lowpriority>.
Job <276416> is submitted to default queue <lowpriority>.
Job <276417> is submitted to default queue <lowpriority>.
Job <276418> is submitted to default queue <lowpriority>.
Job <276419> is submitted to default queue <lowpriority>.
Job <276420> is submitted to default queue <lowpriority>.
Job <276421> is submitted to default queue <lowpriority>.
Job <276422> is submitted to default queue <lowpriority>.
Job <276423> is submitted to default queue <lowpriority>.
Job <276424> is submitted to default queue <lowpriority>.
Job <276425> is submitted to default queue <lowpriority>.
Job <276426> is submitted to default queue <lowpriority>.
Job <276427> is submitted to default queue <lowpriority>.
Job <276428> is submitted to default queue <lowpriority>.
Job <276429> is submitted to default queue <lowpriority>.
Job <276430> is submitted to default queue <lowpriority>.
Job <276431> i

Job <276556> is submitted to default queue <lowpriority>.
Job <276557> is submitted to default queue <lowpriority>.
Job <276558> is submitted to default queue <lowpriority>.
Job <276559> is submitted to default queue <lowpriority>.
Job <276560> is submitted to default queue <lowpriority>.
Job <276561> is submitted to default queue <lowpriority>.
Job <276562> is submitted to default queue <lowpriority>.
Job <276563> is submitted to default queue <lowpriority>.
Job <276564> is submitted to default queue <lowpriority>.
Job <276565> is submitted to default queue <lowpriority>.
Job <276566> is submitted to default queue <lowpriority>.
Job <276567> is submitted to default queue <lowpriority>.
Job <276568> is submitted to default queue <lowpriority>.
Job <276569> is submitted to default queue <lowpriority>.
Job <276570> is submitted to default queue <lowpriority>.
Job <276571> is submitted to default queue <lowpriority>.
Job <276572> is submitted to default queue <lowpriority>.
Job <276573> i

Job <276698> is submitted to default queue <lowpriority>.
Job <276699> is submitted to default queue <lowpriority>.
Job <276700> is submitted to default queue <lowpriority>.
Job <276701> is submitted to default queue <lowpriority>.
Job <276702> is submitted to default queue <lowpriority>.
Job <276703> is submitted to default queue <lowpriority>.
Job <276704> is submitted to default queue <lowpriority>.
Job <276705> is submitted to default queue <lowpriority>.
Job <276706> is submitted to default queue <lowpriority>.
Job <276707> is submitted to default queue <lowpriority>.
Job <276708> is submitted to default queue <lowpriority>.
Job <276709> is submitted to default queue <lowpriority>.
Job <276710> is submitted to default queue <lowpriority>.
Job <276711> is submitted to default queue <lowpriority>.
Job <276712> is submitted to default queue <lowpriority>.
Job <276713> is submitted to default queue <lowpriority>.
Job <276714> is submitted to default queue <lowpriority>.
Job <276715> i

Job <276840> is submitted to default queue <lowpriority>.
Job <276841> is submitted to default queue <lowpriority>.
Job <276842> is submitted to default queue <lowpriority>.
Job <276843> is submitted to default queue <lowpriority>.
Job <276844> is submitted to default queue <lowpriority>.
Job <276845> is submitted to default queue <lowpriority>.
Job <276846> is submitted to default queue <lowpriority>.
Job <276847> is submitted to default queue <lowpriority>.
Job <276848> is submitted to default queue <lowpriority>.
Job <276849> is submitted to default queue <lowpriority>.
Job <276850> is submitted to default queue <lowpriority>.
Job <276851> is submitted to default queue <lowpriority>.
Job <276852> is submitted to default queue <lowpriority>.
Job <276853> is submitted to default queue <lowpriority>.
Job <276854> is submitted to default queue <lowpriority>.
Job <276855> is submitted to default queue <lowpriority>.
Job <276856> is submitted to default queue <lowpriority>.
Job <276857> i